<a href="https://colab.research.google.com/github/khushboogupta13/Inter-IIT-Tech-Meet/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 **SENTIMENT ANALYSIS ON TWEETS** 

In [83]:
data_tweets = pd.read_excel("/content/drive/MyDrive/Inter IIT Tech Meet dataset/dev_data_tweet.xlsx")
data_tweets

,Tweet_ID,Tweet,Mobile_Tech_Tag
0,tweet_0001,You'll 💜 my #PitchWars book if you like: 🦋 hat...,0
1,tweet_0002,RT @SkySportsNews: 🚨 Breaking: #WBA have reach...,0
2,tweet_0003,RT @stealyoman_cuso: really says a lot about s...,0
3,tweet_0004,RT @PGtzsche1: HPV vaccines increased serious ...,0
4,tweet_0005,Ramaphosa says if you are positive you must se...,0
...,...,...,...
3995,tweet_3996,RT @ZeeNews: चीनी मोबाइल कंपनी रियलमी ने अपना ...,1
3996,tweet_3997,RT @ZeeNews: चीनी मोबाइल कंपनी रियलमी ने अपना ...,1
3997,tweet_3998,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1
3998,tweet_3999,QT @AnjaliSingh_IN: Aapne phone hi galat choos...,1


In [84]:
data_tweets = data_tweets.drop_duplicates(subset ="Tweet")
print(len(data_tweets))

2400


In [85]:
#using only the tweets with tag=1 for the start
data_tweets = data_tweets.loc[data_tweets["Mobile_Tech_Tag"] == 1]
data_tweets

,Tweet_ID,Tweet,Mobile_Tech_Tag
3000,tweet_3001,#Samsung is now making another addition into i...,1
3001,tweet_3002,QT @AwamiWeb: Want my hands on #GalaxyS21 🥺 ; ...,1
3002,tweet_3003,QT @AwamiWeb: This is a next level camera 🤩 #G...,1
3003,tweet_3004,Trick to get Google Home Mini at 499 Only With...,1
3004,tweet_3005,QT @MadhavSheth1: We are the pioneers of 64MP ...,1
...,...,...,...
3991,tweet_3992,#Realme ने 5G सेगमेंट में एक धांसू फोन #Realme...,1
3992,tweet_3993,"भारत में लॉन्च हुआ Xiaomi Mi 10i, यह है 2021 क...",1
3997,tweet_3998,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1
3998,tweet_3999,QT @AnjaliSingh_IN: Aapne phone hi galat choos...,1


In [86]:
tweets = list(data_tweets["Tweet"])
print(len(tweets))

569


**Data PreProcessing**

In [87]:
#for translating the hindi tweets to english
!pip install google_trans_new

In [88]:
from google_trans_new import google_translator

def translate_text(sentence):
  return translator.translate(sentence,lang_tgt='en') 

translator = google_translator()
translated_tweets = [translate_text(i) for i in tweets]


In [89]:
data_tweets["Tweet"] = translated_tweets
data_tweets

,Tweet_ID,Tweet,Mobile_Tech_Tag
3000,tweet_3001,#Samsung is now making another addition into i...,1
3001,tweet_3002,QT @AwamiWeb: Want my hands on #GalaxyS21 🥺 ; ...,1
3002,tweet_3003,QT @AwamiWeb: This is a next level camera 🤩 #G...,1
3003,tweet_3004,Trick to get Google Home Mini at 499 Only With...,1
3004,tweet_3005,QT @MadhavSheth1: We are the pioneers of 64MP ...,1
...,...,...,...
3991,tweet_3992,#Realme has launched a Dhansu Phone # Realmev1...,1
3992,tweet_3993,"Xiaomi Mi 10i launched in India, it is the che...",1
3997,tweet_3998,Chinese mobile company Realmi has launched its...,1
3998,tweet_3999,Qt @anjalisingh_in: aapne phone hi galat choos...,1


In [90]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def data_preprocessing(df):
  #lowercasing
  df['Tweet'] = df['Tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))

  #removing the twitter handles and "qt" and retweets
  tweets = list(df["Tweet"])
  for i in range(len(tweets)):
    r = re.findall("@[\w]*",tweets[i])
    for j in r:
      tweets[i] = re.sub(j, '', tweets[i])
    tweets[i] = tweets[i].replace("qt","")
    tweets[i] = tweets[i].replace("rt","")
    tweets[i] = re.sub(r'[^\w\s]', '', tweets[i]) 
  
  df["Tweet"] = tweets

  #removing the urls
  df["Tweet"] = df["Tweet"].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
  df["Tweet"] = df["Tweet"].apply(lambda x: re.sub(r'http?:\/\/\S+', '', x))
  df["Tweet"] = df["Tweet"].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))

  #removing the non letter characters
  df["Tweet"] = df["Tweet"].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))

  df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

  #tokenization
  tokenized_tweet = df["Tweet"].apply(lambda x: nltk.word_tokenize(x))

  #stemming
  tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])

  df["Tweet"] = tokenized_tweet

  
data_preprocessing(data_tweets) 
data_tweets

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Tweet_ID,Tweet,Mobile_Tech_Tag
3000,tweet_3001,"[samsung, make, anoth, addit, into, seri, word...",1
3001,tweet_3002,"[want, hand, galaxi, samsung, galaxi, ultra, m...",1
3002,tweet_3003,"[thi, next, level, camera, galaxi, samsung, ga...",1
3003,tweet_3004,"[trick, googl, home, mini, onli, without, time...",1
3004,tweet_3005,"[pioneer, smaphon, camera, technolog, india, r...",1
...,...,...,...
3991,tweet_3992,"[realm, launch, dhansu, phone, realmev, segmen...",1
3992,tweet_3993,"[xiaomi, launch, india, cheapest, phone, https...",1
3997,tweet_3998,"[chines, mobil, compani, realmi, launch, realm...",1
3998,tweet_3999,"[aapn, phone, galat, choos, liya, haisorri, ph...",1


**Sentiment Analysis**

In [91]:
from textblob import TextBlob

maxx = 0
tweets = list(data_tweets["Tweet"])
for i in range(len(tweets)):
  maxx=max(maxx,len(tweets[i]))
  tweets[i] = " ".join(tweets[i])

data_tweets["Tweet"] = tweets

print(maxx)
print(tweets)


52
['samsung make anoth addit into seri word that upcom variant galaxi awesom batteri charger with brilliant spec more tech updat follow smaphon mobil mobilephon mobilephoto httpstcobuozmc', 'want hand galaxi samsung galaxi ultra most premium smaphon with strong focu camera httpstcotxpysckmz', 'thi next level camera galaxi samsung galaxi ultra most premium smaphon with strong focu camera httpstcotxpysckmz', 'trick googl home mini onli without timesprim coupon first just open googl assist your android smaphon googl assist will open speak googl play googl home mini game googl will show game page like below with httpstcoiqebabk', 'pioneer smaphon camera technolog india realmexpro realmexpro pioneer smaphon camera technolog india giveaway repli use realmexpro answer correctli xperiencethefutur httpstcodvehgqv', 'pioneer smaphon camera technolog india realmexpro pioneer smaphon camera technolog india giveaway repli use realmexpro answer correctli xperiencethefutur httpstcodvehgqv', 'bestest

**Text analysis using Textblob**

In [92]:
from textblob import TextBlob

def sentiment_analysis(df):

  def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

  def getPolarity(text):
    return TextBlob(text).sentiment.polarity
    
  df['TextBlob_Subjectivity'] = df['Tweet'].apply(getSubjectivity)
  df['TextBlob_Polarity'] = df['Tweet'].apply(getPolarity)

  def getAnalysis(score):
    if score < 0:
      return 'Negative'
    else:
      return 'Positive'

  df['TextBlob_Analysis'] = df['TextBlob_Polarity'].apply(getAnalysis )

  return df

In [93]:
data_tweets = sentiment_analysis(data_tweets)
data_tweets.head()

,Tweet_ID,Tweet,Mobile_Tech_Tag,TextBlob_Subjectivity,TextBlob_Polarity,TextBlob_Analysis
3000,tweet_3001,samsung make anoth addit into seri word that u...,1,0.750000,0.700000,Positive
3001,tweet_3002,want hand galaxi samsung galaxi ultra most pre...,1,0.616667,0.466667,Positive
3002,tweet_3003,thi next level camera galaxi samsung galaxi ul...,1,0.411111,0.311111,Positive
3003,tweet_3004,trick googl home mini onli without timesprim c...,1,0.426667,-0.110000,Negative
3004,tweet_3005,pioneer smaphon camera technolog india realmex...,1,0.000000,0.000000,Positive


In [97]:
!pip install afinn

     |████████████████████████████████| 61kB 3.4MB/s 
  Created wheel for afinn: filename=afinn-0.1-cp37-none-any.whl size=53451 sha256=e8c99a6aa65c81599f34389c9d3a137f74797a8969abd5e0b481452f029e1e06
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn


**Sentiment Analysis using Afinn**

In [123]:
from afinn import Afinn

afinn = Afinn()
afinn_scores = [afinn.score(tweet) for tweet in data_tweets['Tweet']]
data_tweets['afinn'] = afinn_scores

sentiment=[]
for score in afinn_scores:
  if score>0:
    sentiment.append(1)
  elif score==0:
    sentiment.append(0)
  else:
    sentiment.append(-1)

data_tweets["sentiments"] = sentiment

data_tweets.head()

,Tweet_ID,Tweet,Mobile_Tech_Tag,TextBlob_Subjectivity,TextBlob_Polarity,TextBlob_Analysis,afinn,sentiments
3000,tweet_3001,samsung make anoth addit into seri word that u...,1,0.750000,0.700000,Positive,4.0,1
3001,tweet_3002,want hand galaxi samsung galaxi ultra most pre...,1,0.616667,0.466667,Positive,3.0,1
3002,tweet_3003,thi next level camera galaxi samsung galaxi ul...,1,0.411111,0.311111,Positive,2.0,1
3003,tweet_3004,trick googl home mini onli without timesprim c...,1,0.426667,-0.110000,Negative,2.0,1
3004,tweet_3005,pioneer smaphon camera technolog india realmex...,1,0.000000,0.000000,Positive,0.0,0


In [124]:
pd.crosstab(data_tweets["sentiments"], data_tweets["afinn"])

afinn,-5.0,-4.0,-3.0,-2.0,-1.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,12.0
sentiments,,,,,,,,,,,,,,,
-1,3,1,2,5,16,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,423,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,30,34,25,12,3,11,2,1,1


In [128]:
#ignoring the tweets which are not relevant
tweets_relevant = data_tweets[data_tweets["sentiments"] != 'not_relevant']

#computing the confusion matrix
confusion = pd.crosstab(np.sign(tweets_relevant["sentiments"]), np.sign(tweets_relevant["afinn"]))
confusion

afinn,-1.0,0.0,1.0
sentiments,,,
-1,27,0,0
0,0,423,0
1,0,0,119


In [129]:
accuracy_3_class = np.sum(np.diag(confusion)) / np.sum(confusion.values)
accuracy_3_class

1.0

In [130]:
confusion_2_class = confusion.iloc[[0, 2], [0, 2]]
confusion_2_class

afinn,-1.0,1.0
sentiments,,
-1,27,0
1,0,119


In [131]:
accuracy_2_class = np.sum(np.diag(confusion_2_class)) / np.sum(confusion_2_class.values)
accuracy_2_class

1.0